<a href="https://colab.research.google.com/github/inesbsilveira/2nd-semester-2023/blob/main/20230911.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install rasterio

import rasterio
from rasterio.windows import Window
import os
import shutil
import numpy as np
from PIL import Image

from fastai.vision.all import *
from sklearn.metrics import confusion_matrix

from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Constants and Parameters

In [6]:
user='I' # escolher 'M' ou 'I'
N_epochs=20
target_names = ['not greenhouse', 'greenhouse']
Nrows,Ncols = 20, 8 # número de blocos a usar 20*8=160

Paths and folders

In [7]:
# Specify the directory path
if user == 'M':
    path = '/content/drive/MyDrive/inv_proj/S2change_DGT/Estufas_Alentejo'
else:
    path = '/content/drive/MyDrive/AAA/'

# Function to create a folder if it doesn't exist
def create_folder(folder_path):
    if os.path.exists(folder_path):
        shutil.rmtree(folder_path)  # Delete the folder and its contents
    os.makedirs(folder_path)  # Create the folder

# Create "labels" folder
folder_name = "labels"
folder_path = os.path.join(path, folder_name)
create_folder(folder_path)

# Create "images" folder
folder_name = "images"
folder_path = os.path.join(path, folder_name)
create_folder(folder_path)

# Create "predicted" folder
folder_name = "predicted"
folder_path = os.path.join(path, folder_name)
create_folder(folder_path)

Functions

In [8]:
def clip_raster_to_subrasters(path, Nrows, Ncols):
    with rasterio.open(path / 'mask_3763_reclass.tif') as src:
        height = src.height // Nrows
        width = src.width // Ncols

        for i in range(Nrows):
            for j in range(Ncols):
                ymin = i * height
                ymax = (i + 1) * height
                xmin = j * width
                xmax = (j + 1) * width
                window = Window.from_slices((ymin, ymax), (xmin, xmax))

                clipped_data = src.read(window=window)
                clipped_meta = src.meta.copy()
                clipped_meta.update({
                    'height': window.height,
                    'width': window.width,
                    'transform': src.window_transform(window)
                })

                output_file = path / 'labels' / f'image_{i}_{j}_mask.png'
                with rasterio.open(output_file, 'w', **clipped_meta) as dst:
                    dst.write(clipped_data)

def rescale_to_8bits(arr):
    arr_std = (arr - np.min(arr)) / (np.max(arr) - np.min(arr)) * 255
    arr_std = arr_std.astype(np.uint8)
    return arr_std

In [9]:
def create_image_dataset(path, Nrows, Ncols):
    with rasterio.open(path / '3bands_clipped_tif.tif') as src:
        height = src.height // Nrows
        width = src.width // Ncols

        for i in range(Nrows):
            for j in range(Ncols):
                ymin = i * height
                ymax = (i + 1) * height
                xmin = j * width
                xmax = (j + 1) * width
                window = Window.from_slices((ymin, ymax), (xmin, xmax))

                clipped_data_1 = rescale_to_8bits(src.read(1, window=window))
                clipped_data_2 = rescale_to_8bits(src.read(2, window=window))
                clipped_data_3 = rescale_to_8bits(src.read(3, window=window))

                rgb_image = np.stack([clipped_data_1, clipped_data_2, clipped_data_3], axis=2)
                image = Image.fromarray(rgb_image, mode='RGB')

                output_file = path / 'images' / f'image_{i}_{j}.png'
                image.save(output_file)

DataBlock and Dataloaders

In [10]:
def create_datablock_and_dataloaders(path):
    def get_items(path):
        image_files = get_image_files(path / 'images')
        return image_files

    def get_mask_fn(image_file):
        return path / 'labels' / f'{image_file.stem}_mask.png'

    items = get_items(path)

    dblock = DataBlock(blocks=(ImageBlock, MaskBlock(target_names)),
                       get_items=get_items,
                       get_y=get_mask_fn,
                       splitter=RandomSplitter(valid_pct=0.2, seed=42))

    dls = dblock.dataloaders(path, bs=8)
    return dls

In [11]:
def train_model(dls, N_epochs):
    learn = unet_learner(dls, resnet34)
    learn.fine_tune(N_epochs)
    return learn

In [12]:
def perform_inference(learn, path):
    test_dl = dls.test_dl(get_image_files(path / 'images'))
    preds = learn.get_preds(dl=test_dl)
    return preds

In [13]:
def save_predicted_masks(preds, path):
    for idx, pred in enumerate(preds[0]):
        sample_image_path = list(test_dl.items)[idx]
        row, column = get_row_column_from_filename(sample_image_path)

        row_folder = path / 'predicted' / f'row_{row}'
        os.makedirs(row_folder, exist_ok=True)

        mask_filename = f'image_{row}_{column}_pred.png'
        mask_filepath = row_folder / mask_filename

        sample_pred = pred.argmax(dim=0).numpy()
        mask_image = Image.fromarray((sample_pred * 255).astype(np.uint8))
        mask_image.save(mask_filepath)

In [14]:
def get_row_column_from_filename(filename):
    filename = os.path.basename(filename)
    parts = filename.split('_')
    row = int(parts[1])
    column = int(parts[2].split('.')[0])
    return row, column

In [15]:
def merge_predicted_masks(path, Nrows, Ncols):
    rows, columns = Nrows, Ncols
    with rasterio.open(path / 'mask_3763_reclass.tif') as src:
        height = src.height // rows
        width = src.width // columns
        merged_width = width * columns
        merged_height = height * rows
        merged_image = Image.new('RGB', (merged_width, merged_height))

        for i in range(rows):
            for j in range(columns):
                mask_file = path / 'predicted' / f'row_{i}' / f'image_{i}_{j}_pred.png'
                mask_image = Image.open(mask_file)

                x_offset = j * width
                y_offset = i * height

                merged_image.paste(mask_image, (x_offset, y_offset))

        merged_output_file = path / 'predicted' / 'merged_predicted_masks_20.png'
        merged_image.save(merged_output_file)


In [ ]:
def main (user, N_epochs, Nrows, Ncols):
  clip_raster_to_subrasters(path)